RDF and OBO to NDEx and Back

This example uses pronto for parsing and generating OBO and RDF files.

In [1]:
import ndex2
import pronto

In [5]:
my_username = 'drh'
my_password = 'drh'
my_ndex_server = 'public.ndexbio.org'

username = drh
password = drh
connected as drh
ndex has 3223 networks


Load the Protein Modification Ontology from an OBO file

In [6]:
print("LOADING PSI-MOD...")
ontology = pronto.Ontology("../files/PSI-MOD.obo")
#ontology = pronto.Ontology("http://purl.obolibrary.org/obo/go/go-basic.obo")
clear_output()
print("DONE!")

DONE!
